In [48]:
!pip install requests
!pip install beautifulsoup4

In [49]:
import requests
from bs4 import BeautifulSoup

Response = requests.get("https://www.honda-indonesia.com/dealers/")
print(Response.status_code)
# print(Response.text)

200


In [50]:
soup = BeautifulSoup(Response.text, 'html.parser')
table_blocks = soup.find_all('div', class_='tw-space-y-4')
print("Number of countries found: ", len(table_blocks))

Number of countries found:  194


In [51]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# --- Request ---
url = "https://www.honda-indonesia.com/dealers/"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
response = requests.get(url, headers=headers, timeout=15)
response.raise_for_status()
print(f"Status: {response.status_code}")

soup = BeautifulSoup(response.text, 'html.parser')

# --- Cari semua konten provinsi (div dengan x-show="open === X") ---
province_content_divs = soup.find_all('div', attrs={'x-show': lambda x: x and 'open ===' in x})

print(f"Jumlah provinsi terdeteksi: {len(province_content_divs)}")

result = []

for content_div in province_content_divs:
    try:
        # 1. Ambil ID provinsi dari x-show
        x_show = content_div.get('x-show')
        province_id = re.search(r'open === (\d+)', x_show).group(1)

        # 2. Cari button provinsi (sebelum content_div)
        button = content_div.find_previous('button')
        if not button:
            continue

        span = button.find('span', class_='tw-font-bold')
        province_name = span.get_text(strip=True) if span else "Unknown"

        # 3. Cari semua blok dealer di dalam content_div
        dealer_blocks = content_div.find_all('div', class_='tw-space-y-4')

        for block in dealer_blocks:
            try:
                # Nama Dealer
                name_tag = block.find('a', href=lambda x: x and '/dealers/' in x)
                dealer = name_tag.get_text(strip=True) if name_tag else "N/A"

                # Alamat
                alamat_tag = block.find('div', class_='tw-text-gray-500')
                alamat = alamat_tag.get_text(strip=True) if alamat_tag else "Tidak tersedia"

                # Koordinat
                direction_tag = None
                for a in block.find_all('a'):
                    if a.get_text(strip=True) == "Direction":
                        direction_tag = a
                        break

                lat, long = "N/A", "N/A"
                if direction_tag and direction_tag.get('href'):
                    href = direction_tag['href']
                    match = re.search(r'q=([-\d.]+),([-\d.]+)', href)
                    if match:
                        lat, long = match.group(1), match.group(2)

                # Simpan
                result.append({
                    'Provinsi': province_name,
                    'Dealer': dealer,
                    'Alamat': alamat,
                    'Latitude': lat,
                    'Longitude': long
                })

            except Exception as e:
                print(f"Error parsing dealer block: {e}")
                continue

    except Exception as e:
        print(f"Error parsing province: {e}")
        continue

# --- Output ---
print("\n" + "="*100)
print("CONTOH 5 DEALER PERTAMA".center(100))
print("="*100)
for item in result[:5]:
    print(f"Provinsi   : {item['Provinsi']}")
    print(f"Dealer     : {item['Dealer']}")
    print(f"Alamat     : {item['Alamat']}")
    print(f"Lat/Long   : {item['Latitude']}, {item['Longitude']}")
    print("-" * 100)


Status: 200
Jumlah provinsi terdeteksi: 32

                                      CONTOH 5 DEALER PERTAMA                                       
Provinsi   : Bali
Dealer     : Honda Bintang Tabanan
Alamat     : 
Lat/Long   : N/A, N/A
----------------------------------------------------------------------------------------------------
Provinsi   : Bali
Dealer     : Honda Cokroaminoto
Alamat     : Jl. Cokroaminoto No. 168
Lat/Long   : N/A, N/A
----------------------------------------------------------------------------------------------------
Provinsi   : Bali
Dealer     : Honda Denpasar Agung
Alamat     : Jl. Hayam Wuruk No. 40
Lat/Long   : -8.65731, 115.22685
----------------------------------------------------------------------------------------------------
Provinsi   : Bali
Dealer     : Honda Dewata Motor
Alamat     : Jl. Imam Bonjol No. 104
Lat/Long   : -8.66767, 115.20567
----------------------------------------------------------------------------------------------------
Provinsi   

In [52]:
# --- Simpan ke Excel ---
import pandas as pd

df = pd.DataFrame(result)

# Add headers
df.columns = ['Provinsi', 'Dealer', 'Alamat', 'Latitude', 'Longitude']

# Save to Excel
df.to_excel('honda_dealers_indonesia.xlsx', index=False)

print(f"Total {len(result)} dealer disimpan ke 'honda_dealers_indonesia.xlsx'")

Total 192 dealer disimpan ke 'honda_dealers_indonesia.xlsx'
